In [2]:
# Use this code snippet in your app.
# If you need more information about configurations
# or implementing the sample code, visit the AWS docs:
# https://aws.amazon.com/developer/language/python/
import json
import boto3
from botocore.exceptions import ClientError


def get_secret(secret_name):
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    
    return secret

In [3]:
S3_CLIENT = boto3.client('s3')

In [4]:
import boto3
import tarfile
from io import BytesIO

# Specify the S3 bucket and object key
bucket_name = 'sagemaker-us-east-1-275461957965'
# object_key = 'huggingface-qlora-2023-10-10-12-58-13-2023-10-10-12-58-17-351/output/model.tar.gz'
object_key = 'huggingface-qlora-2023-10-27-20-47-45-2023-10-27-20-47-46-997/output/model.tar.gz'

# Get the object from S3
obj = S3_CLIENT.get_object(Bucket=bucket_name, Key=object_key)
# Read the file contents
fileobj = obj['Body'].read()

# Specify the directory where you want to extract files
extract_directory = '/home/ubuntu/Llama-customize/model-2'

# Open the file object as a tarfile directly in memory
with tarfile.open(fileobj=BytesIO(fileobj), mode='r:gz') as tar:
    # Extract all the contents of the tar archive into the directory
    tar.extractall(path=extract_directory)

    # List contents
    for member in tar.getmembers():
        print(member.name)

model-00004-of-00014.safetensors
model-00012-of-00014.safetensors
model-00003-of-00014.safetensors
model-00011-of-00014.safetensors
model-00009-of-00014.safetensors
model-00007-of-00014.safetensors
model.safetensors.index.json
model-00014-of-00014.safetensors
tokenizer_config.json
generation_config.json
model-00002-of-00014.safetensors
model-00006-of-00014.safetensors
model-00008-of-00014.safetensors
model-00013-of-00014.safetensors
model-00010-of-00014.safetensors
model-00001-of-00014.safetensors
tokenizer.json
tokenizer.model
config.json
model-00005-of-00014.safetensors
special_tokens_map.json


In [8]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(checkpoint)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)

In [9]:
device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2.self_attn': 0,
 'model.layers.2.mlp.gate_proj': 0,
 'model.layers.2.mlp.up_proj': 0,
 'model.layers.2.mlp.down_proj': 1,
 'model.layers.2.mlp.act_fn': 1,
 'model.layers.2.input_layernorm': 1,
 'model.layers.2.post_attention_layernorm': 1,
 'model.layers.3': 1,
 'model.layers.4.self_attn': 1,
 'model.layers.4.input_layernorm': 2,
 'model.layers.4.post_attention_layernorm': 2,
 'model.layers.5': 2,
 'model.layers.7': 3,
 'model.layers.8': 3,
 'model.layers.9.self_attn': 3,
 'model.layers.9.mlp.gate_proj': 3,
 'model.layers.9.mlp.up_proj': 3,
 'model.layers.9.mlp.down_proj': 'cpu',
 'model.layers.9.mlp.act_fn': 'cpu',
 'model.layers.9.input_layernorm': 'cpu',
 'model.layers.9.post_attention_layernorm': 'cpu',
 'model.layers.10': 'cpu',
 'model.layers.11': 'cpu',
 'model.layers.12': 'cpu',
 'model.layers.13': 'cpu',
 'model.layers.14': 'cpu',
 'model.layers.15': 'cpu',
 'model.layers.16': 'cpu',
 'model.

In [1]:
import json 
import numpy as np
import pandas as pd
tests = ["arc", "hellaswag", "truthfulqa", "mmlu", "winogrande", "gsm8k", "drop"]
models = ['llama_cust_1010', 'llama_cust2', 'llama_hf'] 

_arc = lambda x: x['results']['arc_challenge']['acc_norm']
_hellaswag = lambda x: x['results']['hellaswag']['acc_norm']
_truthfulqa = lambda x: x['results']['truthfulqa_mc']['mc2']
_winogrande = lambda x: x['results']['xwinograd_en']['acc']
_gsm8k =  lambda x: x['results']['gsm8k']['acc']
_drop = lambda x: x['results']['drop']['f1']
_mmlu = lambda data: np.mean([x['acc'] for x in data['results'].values()])


mapping = {
    "arc" : _arc, 
    "hellaswag": _hellaswag,
    "truthfulqa": _truthfulqa,
    "mmlu": _mmlu,
    "winogrande": _winogrande,
    "gsm8k": _gsm8k,
    "drop": _drop
}

results = []
for model in models:
    row = []
    for test in tests: 
        path = f'/home/ubuntu/Llama-customize/evaluate/results/{model}-{test}_results.json'
        with open(path, 'r') as file:
            data = json.load(file)
        row.append(mapping[test](data))
    results.append(row)

df = pd.DataFrame(results, columns=tests, index=models)
df.loc['llama-lb'] = [0.5904, 0.8194,  0.4412, 0.5464, 0.7451, 0.1524, 0.2387]
df

,arc,hellaswag,truthfulqa,mmlu,winogrande,gsm8k,drop
llama_cust_1010,0.570819,0.806712,0.446304,0.525448,0.896774,0.196361,0.368429
llama_cust2,0.564846,0.813185,0.460069,0.536460,0.899785,0.245641,0.289183
llama_hf,0.591297,0.819359,0.439628,0.546256,0.897204,0.324488,0.112190
llama-lb,0.590400,0.819400,0.441200,0.546400,0.745100,0.152400,0.238700


In [24]:
path = f'/home/ubuntu/Llama-customize/evaluate/results/llama_cust_1010-mmlu_results.json'
with open(path, 'r') as file:
    data = json.load(file)

In [30]:
accs = [x['acc'] for x in data['results'].values()]
accs

[0.3,
 0.4962962962962963,
 0.5394736842105263,
 0.48,
 0.5132075471698113,
 0.5694444444444444,
 0.36,
 0.46,
 0.28,
 0.4277456647398844,
 0.35294117647058826,
 0.7,
 0.4127659574468085,
 0.2982456140350877,
 0.47586206896551725,
 0.328042328042328,
 0.29365079365079366,
 0.33,
 0.6129032258064516,
 0.39901477832512317,
 0.6,
 0.6606060606060606,
 0.6767676767676768,
 0.7098445595854922,
 0.4666666666666667,
 0.2962962962962963,
 0.4957983193277311,
 0.33112582781456956,
 0.7174311926605504,
 0.36574074074074076,
 0.7009803921568627,
 0.6751054852320675,
 0.6591928251121076,
 0.5801526717557252,
 0.743801652892562,
 0.6851851851851852,
 0.6012269938650306,
 0.35714285714285715,
 0.6990291262135923,
 0.7521367521367521,
 0.58,
 0.7369093231162197,
 0.5809248554913294,
 0.2569832402234637,
 0.6078431372549019,
 0.6045016077170418,
 0.5802469135802469,
 0.36879432624113473,
 0.37222946544980445,
 0.45588235294117646,
 0.4918300653594771,
 0.6181818181818182,
 0.6285714285714286,
 0.71144